# Data Visualization - Trực quan hóa dữ liệu

**Người phụ trách:** Người 4 - Data Visualization

## Mục tiêu:
- Trực quan hóa dữ liệu với Seaborn, Plotly
- Tạo biểu đồ tương tác
- Thiết kế dashboard tổng hợp



In [2]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Đọc dữ liệu
df = pd.read_csv('../data/foodpanda_orders.csv')



ModuleNotFoundError: No module named 'seaborn'

In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Data Visualization – Người 4\n",
        "\n",
        "Notebook này thực hiện trực quan hóa dữ liệu cho dự án Foodpanda Analytics.\n",
        "\n",
        "## Mục tiêu:\n",
        "- Vẽ biểu đồ trực quan hóa bằng Matplotlib & Seaborn\n",
        "- Biểu đồ tương tác bằng Plotly\n",
        "- Histogram, Boxplot, Heatmap, Scatterplot, Time Series\n",
        "- Tạo mini dashboard tổng hợp\n"
      ]
    },

    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import pandas as pd\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns\n",
        "import plotly.express as px\n",
        "import plotly.graph_objects as go\n",
        "from plotly.subplots import make_subplots\n",
        "\n",
        "plt.style.use('ggplot')\n",
        "\n",
        "df = pd.read_csv('../data/foodpanda_orders.csv')\n",
        "df['order_date'] = pd.to_datetime(df['order_date'])\n",
        "df.head()"
      ]
    },

    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Histogram – Phân phối giá trị đơn hàng"
      ]
    },

    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "plt.figure(figsize=(8,5))\n",
        "sns.histplot(df['total_amount'], kde=True)\n",
        "plt.title('Phân phối giá trị đơn hàng')\n",
        "plt.show()"
      ]
    },

    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Boxplot – So sánh biến phân phối"
      ]
    },

    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "plt.figure(figsize=(8,5))\n",
        "sns.boxplot(x=df['total_amount'])\n",
        "plt.title('Boxplot giá trị đơn hàng')\n",
        "plt.show()"
      ]
    },

    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Scatterplot – Quan hệ giữa thời gian giao hàng & tổng tiền"
      ]
    },

    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "plt.figure(figsize=(8,5))\n",
        "sns.scatterplot(x='delivery_time', y='total_amount', data=df, alpha=0.5)\n",
        "plt.title('Delivery time vs Total amount')\n",
        "plt.xlabel('Thời gian giao hàng (phút)')\n",
        "plt.ylabel('Giá trị đơn hàng (VND)')\n",
        "plt.show()"
      ]
    },

    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Heatmap – Ma trận tương quan"
      ]
    },

    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "plt.figure(figsize=(8,6))\n",
        "sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')\n",
        "plt.title('Correlation Heatmap')\n",
        "plt.show()"
      ]
    },

    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Time Series – Doanh thu theo ngày"
      ]
    },

    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "daily_rev = df.groupby(df['order_date'].dt.date)['total_amount'].sum()\n",
        "\n",
        "plt.figure(figsize=(12,5))\n",
        "plt.plot(daily_rev)\n",
        "plt.title('Doanh thu theo ngày')\n",
        "plt.xlabel('Ngày')\n",
        "plt.ylabel('Doanh thu')\n",
        "plt.xticks(rotation=45)\n",
        "plt.show()"
      ]
    },

    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Interactive Visualization – Plotly\n",
        "Biểu đồ tương tác phù hợp khi người xem muốn zoom, hover, xem giá trị chính xác."
      ]
    },

    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "fig = px.scatter(df, x='delivery_time', y='total_amount', title='Interactive Scatter: Delivery vs Amount', opacity=0.6)\n",
        "fig.show()"
      ]
    },

    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "fig = px.line(x=daily_rev.index, y=daily_rev.values, title='Interactive Time Series – Revenue per Day')\n",
        "fig.show()"
      ]
    },

    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Dashboard Mini – Tổng hợp nhiều biểu đồ Plotly"
      ]
    },

    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "fig = make_subplots(rows=2, cols=2, subplot_titles=('Histogram', 'Boxplot', 'Scatter', 'Daily Revenue'))\n",
        "\n",
        "# Histogram\n",
        "fig.add_trace(go.Histogram(x=df['total_amount']), row=1, col=1)\n",
        "\n",
        "# Boxplot\n",
        "fig.add_trace(go.Box(x=df['total_amount']), row=1, col=2)\n",
        "\n",
        "# Scatter\n",
        "fig.add_trace(go.Scatter(x=df['delivery_time'], y=df['total_amount'], mode='markers', opacity=0.5), row=2, col=1)\n",
        "\n",
        "# Time series\n",
        "fig.add_trace(go.Scatter(x=daily_rev.index, y=daily_rev.values, mode='lines'), row=2, col=2)\n",
        "\n",
        "fig.update_layout(height=800, width=1000, title_text='📊 Dashboard Data Visualization')\n",
        "fig.show()"
      ]
    },

    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Kết luận\n",
        "- Biểu đồ cho thấy phân phối đơn hàng lệch phải (right-skewed)\n",
        "- Có sự tương quan nhẹ giữa thời gian giao hàng và tổng tiền\n",
        "- Doanh thu biến động theo ngày rõ rệt → phù hợp cho dự báo (forecasting)\n",
        "- Dashboard cung cấp cái nhìn toàn diện cho nhóm nghiên cứu\n"
      ]
    }
  ],

  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.10"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 2
}

NameError: name 'null' is not defined